# Intro to Recommender Systems Lab

Complete the exercises below to solidify your knowledge and understanding of recommender systems.

For this lab, we are going to be putting together a user similarity based recommender system in a step-by-step fashion. Our data set contains customer grocery purchases, and we will use similar purchase behavior to inform our recommender system. Our recommender system will generate 5 recommendations for each customer based on the purchases they have made.

In [1]:
import pandas as pd
from scipy.spatial.distance import pdist, squareform

In [2]:
customer_product_sales = pd.read_csv('../data/customer_product_sales.csv')

In [3]:
customer_product_sales.head()

,CustomerID,FirstName,LastName,SalesID,ProductID,ProductName,Quantity
0,61288,Rosa,Andersen,134196,229,Bread - Hot Dog Buns,16
1,77352,Myron,Murray,6167892,229,Bread - Hot Dog Buns,20
2,40094,Susan,Stevenson,5970885,229,Bread - Hot Dog Buns,11
3,23548,Tricia,Vincent,6426954,229,Bread - Hot Dog Buns,6
4,78981,Scott,Burch,819094,229,Bread - Hot Dog Buns,20


In [12]:
customer_product_sales.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 68584 entries, 0 to 68583
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   CustomerID   68584 non-null  int64 
 1   FirstName    68584 non-null  object
 2   LastName     68584 non-null  object
 3   SalesID      68584 non-null  int64 
 4   ProductID    68584 non-null  int64 
 5   ProductName  68584 non-null  object
 6   Quantity     68584 non-null  int64 
dtypes: int64(4), object(3)
memory usage: 3.7+ MB


## Step 1: Create a data frame that contains the total quantity of each product purchased by each customer.

You will need to group by CustomerID and ProductName and then sum the Quantity field.

In [4]:
customer_product_sales_total = customer_product_sales.groupby(by=['CustomerID', 
                                'ProductName'], as_index=0)['Quantity'].sum()
customer_product_sales_total

,CustomerID,ProductName,Quantity
0,33,Apricots - Dried,1
1,33,Assorted Desserts,1
2,33,Bandage - Flexible Neon,1
3,33,"Bar Mix - Pina Colada, 355 Ml",1
4,33,"Beans - Kidney, Canned",1
...,...,...,...
63623,98200,Vol Au Vents,50
63624,98200,Wasabi Powder,25
63625,98200,Wine - Fume Blanc Fetzer,25
63626,98200,Wine - Hardys Bankside Shiraz,25


## Step 2: Use the `pivot_table` method to create a product by customer matrix.

The rows of the matrix should represent the products, the columns should represent the customers, and the values should be the quantities of each product purchased by each customer. You will also need to replace nulls with zeros, which you can do using the `fillna` method.

In [5]:
product_customer_matrix = customer_product_sales_total.pivot_table(values='Quantity', 
                index='ProductName', columns='CustomerID', aggfunc='sum').fillna(0)
product_customer_matrix

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
ProductName,,,,,,,,,,,,,,,,,,,,,
Anchovy Paste - 56 G Tube,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"Appetizer - Mini Egg Roll, Shrimp",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,25.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Appetizer - Mushroom Tart,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,25.0,0.0
Appetizer - Sausage Rolls,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,25.0,25.0,25.0,0.0,25.0,0.0
Apricots - Dried,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Yeast Dry - Fermipan,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0
Yoghurt Tubes,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,25.0,0.0,0.0
"Yogurt - Blueberry, 175 Gr",0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,25.0,0.0,0.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0


## Step 3: Create a customer similarity matrix using `squareform` and `pdist`. For the distance metric, choose "euclidean."

In [6]:
distance = pdist(product_customer_matrix.values.T, metric='euclidean')
distance

array([ 11.91637529,  10.48808848,  11.22497216, ..., 304.13812651,
       305.16389039, 303.10889132])

In [7]:
customer_similarity_matrix = pd.DataFrame(squareform(distance), 
                                          index=product_customer_matrix.columns, 
                                          columns=product_customer_matrix.columns)
customer_similarity_matrix

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
CustomerID,,,,,,,,,,,,,,,,,,,,,
33,0.000000,11.916375,10.488088,11.224972,11.401754,11.090537,12.409674,11.045361,11.269428,11.489125,...,206.871941,213.180675,225.656819,198.232187,230.913404,220.501701,217.188858,228.628520,239.000000,229.773802
200,11.916375,0.000000,11.747340,12.083046,12.569805,12.288206,12.165525,12.083046,11.874342,12.000000,...,206.310446,212.635839,224.697575,197.139544,230.952376,220.202180,215.728997,228.010965,239.037654,229.704158
264,10.488088,11.747340,0.000000,11.489125,11.224972,11.445523,12.000000,11.401754,11.180340,11.747340,...,206.387984,212.946003,225.435135,197.600607,230.371439,219.136943,216.612557,228.081126,238.266657,229.773802
356,11.224972,12.083046,11.489125,0.000000,12.083046,11.789826,12.328828,11.135529,11.958261,12.165525,...,206.649462,213.082144,225.452878,197.494304,231.038958,219.952268,217.437347,228.098663,238.493186,229.464594
412,11.401754,12.569805,11.224972,12.083046,0.000000,11.704700,12.328828,11.135529,11.789826,11.747340,...,206.900942,211.679002,225.572605,197.630969,230.614397,219.733930,217.446545,227.997807,238.396728,228.927936
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97928,220.501701,220.202180,219.136943,219.952268,219.733930,219.599636,219.538152,219.924987,219.827205,220.070443,...,283.945417,283.945417,302.076149,272.717803,278.388218,0.000000,273.861279,291.547595,306.186218,307.205143
98069,217.188858,215.728997,216.612557,217.437347,217.446545,217.425849,216.903204,217.294731,217.080630,216.751009,...,283.945417,283.945417,295.803989,283.945417,285.043856,273.861279,0.000000,287.228132,297.909382,294.745653
98159,228.628520,228.010965,228.081126,228.098663,227.997807,228.197283,228.028507,228.181945,227.868383,228.103047,...,283.945417,279.508497,300.000000,290.473751,300.000000,291.547595,287.228132,0.000000,304.138127,305.163890


In [8]:
customer_similarity_matrix_norm = pd.DataFrame(1/(1 + squareform(pdist(product_customer_matrix.T, 'euclidean'))),
                                index=product_customer_matrix.columns,
                                columns=product_customer_matrix.columns)

customer_similarity_matrix_norm

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
CustomerID,,,,,,,,,,,,,,,,,,,,,
33,1.000000,0.077421,0.087047,0.081800,0.080634,0.082709,0.074573,0.083020,0.081503,0.080070,...,0.004811,0.004669,0.004412,0.005019,0.004312,0.004515,0.004583,0.004355,0.004167,0.004333
200,0.077421,1.000000,0.078448,0.076435,0.073693,0.075255,0.075956,0.076435,0.077674,0.076923,...,0.004824,0.004681,0.004431,0.005047,0.004311,0.004521,0.004614,0.004367,0.004166,0.004335
264,0.087047,0.078448,1.000000,0.080070,0.081800,0.080350,0.076923,0.080634,0.082100,0.078448,...,0.004822,0.004674,0.004416,0.005035,0.004322,0.004543,0.004595,0.004365,0.004179,0.004333
356,0.081800,0.076435,0.080070,1.000000,0.076435,0.078187,0.075025,0.082403,0.077171,0.075956,...,0.004816,0.004671,0.004416,0.005038,0.004310,0.004526,0.004578,0.004365,0.004175,0.004339
412,0.080634,0.073693,0.081800,0.076435,1.000000,0.078711,0.075025,0.082403,0.078187,0.078448,...,0.004810,0.004702,0.004414,0.005034,0.004318,0.004530,0.004578,0.004367,0.004177,0.004349
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97928,0.004515,0.004521,0.004543,0.004526,0.004530,0.004533,0.004534,0.004526,0.004528,0.004523,...,0.003509,0.003509,0.003300,0.003653,0.003579,1.000000,0.003638,0.003418,0.003255,0.003245
98069,0.004583,0.004614,0.004595,0.004578,0.004578,0.004578,0.004589,0.004581,0.004585,0.004592,...,0.003509,0.003509,0.003369,0.003509,0.003496,0.003638,1.000000,0.003469,0.003345,0.003381
98159,0.004355,0.004367,0.004365,0.004365,0.004367,0.004363,0.004366,0.004363,0.004369,0.004365,...,0.003509,0.003565,0.003322,0.003431,0.003322,0.003418,0.003469,1.000000,0.003277,0.003266


In [9]:
customer_similarity_matrix_norm.columns

Index([   33,   200,   264,   356,   412,   464,   477,   639,   649,   669,
       ...
       97697, 97753, 97769, 97793, 97900, 97928, 98069, 98159, 98185, 98200],
      dtype='int64', name='CustomerID', length=1000)

## Step 4: Check your results by generating a list of the top 5 most similar customers for a specific CustomerID.

In [36]:
customer_similarity_matrix_sm = customer_similarity_matrix_norm[200].sort_values(ascending=False).iloc[1:6].index
customer_similarity_matrix_sm

Index([3317, 1072, 3535, 1920, 3909], dtype='int64', name='CustomerID')

## Step 5: From the data frame you created in Step 1, select the records for the list of similar CustomerIDs you obtained in Step 4.

In [24]:
similar_customers = customer_product_sales_total[customer_product_sales_total['CustomerID'].isin(customer_similarity_matrix_sm)]
similar_customers

,CustomerID,ProductName,Quantity
1082,1072,"Appetizer - Mini Egg Roll, Shrimp",1
1083,1072,Appetizer - Sausage Rolls,1
1084,1072,"Beans - Kidney, Canned",1
1085,1072,Beef - Montreal Smoked Brisket,1
1086,1072,Beef - Striploin Aa,1
...,...,...,...
3263,3909,Wine - Charddonnay Errazuriz,2
3264,3909,Wine - Fume Blanc Fetzer,1
3265,3909,"Wine - Magnotta, Merlot Sr Vqa",1
3266,3909,Wine - Toasted Head,1


## Step 6: Aggregate those customer purchase records by ProductName, sum the Quantity field, and then rank them in descending order by quantity.

This will give you the total number of each product purchased by the 5 most similar customers to the customer you selected in order from most purchased to least.

In [25]:
similar_customers_product = similar_customers.groupby('ProductName')['Quantity'].sum().sort_values(ascending=False).reset_index()
similar_customers_product

,ProductName,Quantity
0,Soup - Campbells Bean Medley,4
1,Muffin - Carrot Individual Wrap,3
2,Bay Leaf,3
3,Pork - Kidney,3
4,"Pepper - Black, Whole",3
...,...,...
202,Kiwi,1
203,Knife Plastic - White,1
204,Lambcasing,1
205,"Lentils - Red, Dry",1


## Step 7: Filter the list for products that the chosen customer has not yet purchased and then recommend the top 5 products with the highest quantities that are left.

- Merge the ranked products data frame with the customer product matrix on the ProductName field.
- Filter for records where the chosen customer has not purchased the product.
- Show the top 5 results.

In [27]:
merged_ranked_products = similar_customers_product.merge(product_customer_matrix.reset_index()[['ProductName', 200]])
merged_ranked_products

,ProductName,Quantity,200
0,Soup - Campbells Bean Medley,4,0.0
1,Muffin - Carrot Individual Wrap,3,0.0
2,Bay Leaf,3,0.0
3,Pork - Kidney,3,0.0
4,"Pepper - Black, Whole",3,1.0
...,...,...,...
202,Kiwi,1,1.0
203,Knife Plastic - White,1,0.0
204,Lambcasing,1,0.0
205,"Lentils - Red, Dry",1,1.0


In [28]:
merged_ranked_products[merged_ranked_products[200] == 0].head(5).reset_index()

,index,ProductName,Quantity,200
0,0,Soup - Campbells Bean Medley,4,0.0
1,1,Muffin - Carrot Individual Wrap,3,0.0
2,2,Bay Leaf,3,0.0
3,3,Pork - Kidney,3,0.0
4,6,Wanton Wrap,3,0.0


## Step 8: Now that we have generated product recommendations for a single user, put the pieces together and iterate over a list of all CustomerIDs.

- Create an empty dictionary that will hold the recommendations for all customers.
- Create a list of unique CustomerIDs to iterate over.
- Iterate over the customer list performing steps 4 through 7 for each and appending the results of each iteration to the dictionary you created.

In [45]:
customers_recommendations = {'CustomerID': [],'Product_01': [], 'Product_02': [],
                       'Product_03': [], 'Product_04': [], 'Product_05': []}
unique_customers = list(customer_product_sales['CustomerID'].unique())

for i in unique_customers:
    customer_similarity_matrix_sm = customer_similarity_matrix_norm[i].sort_values(ascending=False).iloc[1:6].index
    similar_customers = customer_product_sales_total[customer_product_sales_total['CustomerID'].isin(customer_similarity_matrix_sm)]
    similar_customers_product = similar_customers.groupby('ProductName')['Quantity'].sum().sort_values(ascending=False).reset_index()
    merged_ranked_products = similar_customers_product.merge(product_customer_matrix.reset_index()[['ProductName', i]])
    merged_ranked_products[merged_ranked_products[i] == 0].head(5).reset_index()
    
    customers_recommendations['CustomerID'].append(i)
    customers_recommendations['Product_01'].append(merged_ranked_products['ProductName'][0])
    customers_recommendations['Product_02'].append(merged_ranked_products['ProductName'][1])
    customers_recommendations['Product_03'].append(merged_ranked_products['ProductName'][2])
    customers_recommendations['Product_04'].append(merged_ranked_products['ProductName'][3])
    customers_recommendations['Product_05'].append(merged_ranked_products['ProductName'][4])

##  Step 9: Store the results in a Pandas data frame. The data frame should a column for Customer ID and then a column for each of the 5 product recommendations for each customer.

In [46]:
product_recommendations = pd.DataFrame(customers_recommendations)
product_recommendations

,CustomerID,Product_01,Product_02,Product_03,Product_04,Product_05
0,61288,Beef - Rib Eye Aaa,Jagermeister,Chicken - Soup Base,Lime Cordial - Roses,"Lamb - Pieces, Diced"
1,77352,Wine - Valpolicella Masi,Fuji Apples,Tahini Paste,Ocean Spray - Ruby Red,Tuna - Salad Premix
2,40094,Bread - Italian Corn Meal Poly,"Wine - White, Colubia Cresh",Puree - Mocha,"Cheese - Brie,danish",Cookie - Dough Variety
3,23548,Tea - English Breakfast,Squid - Tubes / Tenticles 10/20,Spinach - Baby,Jolt Cola - Electric Blue,Sprouts - Baby Pea Tendrils
4,78981,Isomalt,Crush - Cream Soda,Vanilla Beans,Cassis,Tea - Earl Grey
...,...,...,...,...,...,...
995,49005,Pickerel - Fillets,Phyllo Dough,"Pasta - Penne, Rigate, Dry",Lettuce - Spring Mix,Browning Caramel Glace
996,41286,Beef - Ground Medium,Veal - Inside,Beer - Labatt Blue,"Thyme - Lemon, Fresh",Wonton Wrappers
997,85878,"Beef - Tenderlion, Center Cut",Cake - Mini Cheesecake,Olives - Kalamata,Wine - White Cab Sauv.on,Yoghurt Tubes
998,68506,Onions - Cippolini,"Veal - Brisket, Provimi,bnls",Beef - Ground Medium,Scampi Tail,"Rosemary - Primerba, Paste"


## Step 10: Change the distance metric used in Step 3 to something other than euclidean (correlation, cityblock, consine, jaccard, etc.). Regenerate the recommendations for all customers and note the differences.

In [47]:
distance = pdist(product_customer_matrix.values.T, metric='cityblock')
customer_similarity_matrix = pd.DataFrame(squareform(distance), 
                                          index=product_customer_matrix.columns, 
                                          columns=product_customer_matrix.columns)
customer_similarity_matrix_norm = pd.DataFrame(1/(1 + squareform(pdist(product_customer_matrix.T, 'cityblock'))),
                                index=product_customer_matrix.columns,
                                columns=product_customer_matrix.columns)

In [48]:
customers_recommendations = {'CustomerID': [],'Product_01': [], 'Product_02': [],
                       'Product_03': [], 'Product_04': [], 'Product_05': []}
unique_customers = list(customer_product_sales['CustomerID'].unique())

for i in unique_customers:
    customer_similarity_matrix_sm = customer_similarity_matrix_norm[i].sort_values(ascending=False).iloc[1:6].index
    similar_customers = customer_product_sales_total[customer_product_sales_total['CustomerID'].isin(customer_similarity_matrix_sm)]
    similar_customers_product = similar_customers.groupby('ProductName')['Quantity'].sum().sort_values(ascending=False).reset_index()
    merged_ranked_products = similar_customers_product.merge(product_customer_matrix.reset_index()[['ProductName', i]])
    merged_ranked_products[merged_ranked_products[i] == 0].head(5).reset_index()
    
    customers_recommendations['CustomerID'].append(i)
    customers_recommendations['Product_01'].append(merged_ranked_products['ProductName'][0])
    customers_recommendations['Product_02'].append(merged_ranked_products['ProductName'][1])
    customers_recommendations['Product_03'].append(merged_ranked_products['ProductName'][2])
    customers_recommendations['Product_04'].append(merged_ranked_products['ProductName'][3])
    customers_recommendations['Product_05'].append(merged_ranked_products['ProductName'][4])

In [50]:
product_recommendations = pd.DataFrame(customers_recommendations)
product_recommendations

,CustomerID,Product_01,Product_02,Product_03,Product_04,Product_05
0,61288,Soup - Campbells Bean Medley,Knife Plastic - White,Sausage - Liver,Butter - Unsalted,Bay Leaf
1,77352,Butter - Unsalted,Tia Maria,Wine - Blue Nun Qualitatswein,Juice - Lime,"Sole - Dover, Whole, Fresh"
2,40094,"Pepper - Black, Whole",Knife Plastic - White,Butter - Unsalted,Wine - Ej Gallo Sierra Valley,Wine - Blue Nun Qualitatswein
3,23548,Beef - Texas Style Burger,"Wine - White, Colubia Cresh",Pork - Kidney,Knife Plastic - White,Squid - Tubes / Tenticles 10/20
4,78981,Butter - Unsalted,Veal - Sweetbread,"Veal - Inside, Choice",Beef - Montreal Smoked Brisket,Tia Maria
...,...,...,...,...,...,...
995,49005,Cod - Black Whole Fillet,Kellogs Special K Cereal,Oil - Shortening - All - Purpose,"Salsify, Organic",Curry Paste - Madras
996,41286,Pork - Kidney,"Cheese - Boursin, Garlic / Herbs",Butter - Unsalted,Ice Cream Bar - Hageen Daz To,Knife Plastic - White
997,85878,Juice - Lime,Sobe - Tropical Energy,Ice Cream Bar - Hageen Daz To,Knife Plastic - White,Soup - Campbells Tomato Ravioli
998,68506,Squid - Tubes / Tenticles 10/20,"Pepper - Black, Whole",Pork - Hock And Feet Attached,Halibut - Steaks,Cod - Black Whole Fillet
